# Import Packages



In [ ]:
import pandas as pd
import numpy as np
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from json import loads, dumps

# Create Interceptor

In [ ]:
USER_AGENT = '"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"'
SEC_CH_UA = '\"Google Chrome\";v=\"123\", \"Not:A-Brand\";v=\"8\", \"Chromium\";v=\"123\"'
REFERER = "https://www.google.com"

def interceptor(request):
    del request.header["user-agent"]
    request.header["user-agent"] = USER_AGENT
    request.header["sec-ch-ua"] = SEC_CH_UA
    request.header["referer"] = REFERER

# Decompose function to find corams


In [ ]:
def decompose(text):
    if "\n" in text:
        return text.split("\n")[1].split(" ")
    return text.split(" ")

# Extract coram names

In [ ]:
def extract_names(text):
    remove_words_containing = ('CJ', 'AG', 'J', 'DCJ', 'JA', 'AR', 'JC', 'SAR', 'JCA', 'SJ', 'JAD', 'and',
                               'CJ,', 'AG,', 'J,', 'DCJ,', 'JA,', 'AR,', 'JC,', 'SAR,', 'JCA,', 'SJ,', 'JAD,')
    name = ""
    name_list = []

    for word in text:
        if word not in remove_words_containing:
            name += word + " "
        else:
            if len(name):
                name_list.append(name.strip())
                name = ""
    return name_list

# Coram Extraction (2000 - 2015)

In [ ]:
# Coram Extraction (2000 to 2015)
# # Creating the webdriver
# driver = webdriver.Chrome()
# driver.request_interceptor = interceptor
# driver.maximize_window()

# base_url = "https://www.elitigation.sg/gdviewer/s/{year}_{court}_{case_id}"

# years = range(2000, 2016)
# courts = ['SGHC', 'SGCA']
# case_ids = range(1, 430)

# for year in years:
#     for court in courts:
#         for case_id in case_ids:
#             case_url = base_url.format(year=year, court=court, case_id=case_id)
#             driver.get(case_url)
#             containers = driver.find_elements(By.XPATH, '')

# Coram Extraction (2016 to 2023)

In [ ]:
# Creating the webdriver
driver = webdriver.Chrome()
driver.request_interceptor = interceptor
driver.maximize_window()

base_url = "https://www.elitigation.sg/gdviewer/s/{year}_{court}_{case_id}"

years = range(2016, 2024)
courts = ['SGHC', 'SGCA']
case_ids = range(1, 430)
SPECIAL_CASES = (31, 88)

temp_df = pd.DataFrame(columns=["casename", "coram"])
counter = 0
for year in years:
    for court in courts:
        for case_id in case_ids:
            case_url = base_url.format(year=year, court=court, case_id=case_id)
            driver.get(case_url)
            containers = driver.find_elements(By.CSS_SELECTOR, 'div.HN-Coram.text-left')
            if not containers:
                containers = driver.find_elements(By.CSS_SELECTOR, 'div.txt-body.text-left')
            if not containers:
                containers = driver.find_elements(By.CSS_SELECTOR, 'div.Judg-Date-Reserved.text-left')
            coram_list_unprocessed = list(map(lambda x: x.text, containers))
            print(coram_list_unprocessed)
            if len(coram_list_unprocessed) == 1 or case_id in SPECIAL_CASES:   
                coram_list_processed = coram_list_unprocessed[0].split("\n")[1] # 2nd element contains the coram
            else:
                coram_list_processed = coram_list_unprocessed[1]
            # print(coram_list_processed)
            # print(decompose(coram_list_processed))
            print(counter, extract_names(decompose(coram_list_processed)))
            temp_df.loc[len(temp_df.index)] = [f"{year}_{court}_{case_id}", extract_names(decompose(coram_list_processed))]
            counter += 1
            sec = random.randint(1, 10)
            time.sleep(sec)
        #     break
        # break

        # Save to json
        file = f"{year}_{court}_coram.json"
        temp_df.to_json(file, orient="records", lines=True)

driver.quit()
 
temp_df